# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../data/')

In [2]:
import player_data as player
import pandas as pd
import patsy
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

# from sklearn.preprocessing import StandardScaler

# from sklearn.model_selection import train_test_split, GridSearchCV

/home/tom/footbot/venv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Getting data

In [3]:
file_name = 'csvs/element_gameweek_2019-05-14.csv'

In [190]:
element_gameweek_df = pd.read_csv(file_name, index_col=0)

In [191]:
current_event = 11

In [192]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
]

In [193]:
player.add_home_categorical_variable(element_gameweek_df)

In [194]:
element_gameweek_df.head()

,row_id,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,team_a,team_a_difficulty,team_h,team_h_difficulty,element_type,team,web_name,safe_web_name,own_team,opposition_team
0,0,0,42,0,0,0,24,0,2,31,...,13,4,1,4,1,1,Cech,cech,1,13
1,1,0,35,0,0,0,26,0,1,25,...,1,4,6,4,1,1,Cech,cech,1,6
2,2,0,24,0,0,0,17,0,0,20,...,19,4,1,3,1,1,Cech,cech,1,19
3,3,0,38,0,0,0,8,0,0,24,...,1,2,5,4,1,1,Cech,cech,1,5
4,4,0,33,0,0,0,13,0,2,23,...,1,3,15,4,1,1,Cech,cech,1,15


In [195]:
cols = [
    'element',
    'event',
    'own_team',
    'opposition_team',
    'was_home',
    'element_type',
    'value',
    'total_points',
    'minutes'
]

rolling_cols = ['total_points']

In [196]:
element_gameweek_df = element_gameweek_df[cols]
element_gameweek_df['element_fixture_rank'] = element_gameweek_df.groupby('element')['event'].rank()
element_gameweek_df.head()

,element,event,own_team,opposition_team,was_home,element_type,value,total_points,minutes,element_fixture_rank
0,1,1,1,13,1,1,50,3,90,1.0
1,1,2,1,6,0,1,50,3,90,2.0
2,1,3,1,19,1,1,50,3,90,3.0
3,1,4,1,5,0,1,50,1,90,4.0
4,1,5,1,15,0,1,50,2,90,5.0


In [11]:
# some players don't have data for all fixtures, what should we do about it?
#df[df['element'].isin(list(df.groupby('element').size()[df.groupby('element').size() < 10].index))]['minutes'].describe()

# Munging data

## Get player mean points

In [12]:
# copy main df
rolling_player_mean_df = element_gameweek_df.copy()

# average rolling cols over previous events 
rolling_player_mean_df = \
rolling_player_mean_df.groupby('element', as_index=True)[['element_fixture_rank'] + rolling_cols]\
.rolling(38, min_periods=1).mean().reset_index('element')

# increment element fixture rank, as we don't know data ahead of time
rolling_player_mean_df['element_fixture_rank'] =\
rolling_player_mean_df.groupby('element')['element_fixture_rank'].rank() + 1

rolling_player_mean_df.head(10)

,element,element_fixture_rank,total_points
0,1,2.0,3.000000
1,1,3.0,3.000000
2,1,4.0,3.000000
3,1,5.0,2.500000
4,1,6.0,2.400000
5,1,7.0,3.833333
6,1,8.0,3.428571
7,1,9.0,3.000000
8,1,10.0,2.666667
9,1,11.0,2.400000


In [13]:
element_gameweek_df =\
    element_gameweek_df.join(
        rolling_player_mean_df.set_index(['element', 'element_fixture_rank']),
        on=['element', 'element_fixture_rank'],
        rsuffix='_player_mean',
        how='left'
)

element_gameweek_df.head()

,element,event,own_team,opposition_team,was_home,element_type,value,total_points,minutes,element_fixture_rank,total_points_player_mean
0,1,1,1,13,1,1,50,3,90,1.0,NaN
1,1,2,1,6,0,1,50,3,90,2.0,3.0
2,1,3,1,19,1,1,50,3,90,3.0,3.0
3,1,4,1,5,0,1,50,1,90,4.0,3.0
4,1,5,1,15,0,1,50,2,90,5.0,2.5


In [14]:
# what should we do about gameweek 1, where we don't have an average of points?
# does this work for double or missing gameweeks?

## Get player max points

In [15]:
# copy main df
rolling_player_max_df = element_gameweek_df.copy()

# average rolling cols over previous events 
rolling_player_max_df = \
rolling_player_max_df.groupby('element', as_index=True)[['element_fixture_rank'] + rolling_cols]\
.rolling(38, min_periods=1).max().reset_index('element')

# increment element fixture rank, as we don't know data ahead of time
rolling_player_max_df['element_fixture_rank'] =\
rolling_player_max_df.groupby('element')['element_fixture_rank'].rank() + 1

rolling_player_max_df.head(10)

,element,element_fixture_rank,total_points
0,1,2.0,3.0
1,1,3.0,3.0
2,1,4.0,3.0
3,1,5.0,3.0
4,1,6.0,3.0
5,1,7.0,11.0
6,1,8.0,11.0
7,1,9.0,11.0
8,1,10.0,11.0
9,1,11.0,11.0


In [16]:
element_gameweek_df =\
    element_gameweek_df.join(
        rolling_player_max_df.set_index(['element', 'element_fixture_rank']),
        on=['element', 'element_fixture_rank'],
        rsuffix='_player_max',
        how='left'
)

element_gameweek_df.head()

,element,event,own_team,opposition_team,was_home,element_type,value,total_points,minutes,element_fixture_rank,total_points_player_mean,total_points_player_max
0,1,1,1,13,1,1,50,3,90,1.0,NaN,NaN
1,1,2,1,6,0,1,50,3,90,2.0,3.0,3.0
2,1,3,1,19,1,1,50,3,90,3.0,3.0,3.0
3,1,4,1,5,0,1,50,1,90,4.0,3.0,3.0
4,1,5,1,15,0,1,50,2,90,5.0,2.5,3.0


In [17]:
# what should we do about gameweek 1, where we don't have an average of points?
# does this work for double or missing gameweeks?

## Get player min points

In [18]:
# copy main df
rolling_player_min_df = element_gameweek_df.copy()

# average rolling cols over previous events 
rolling_player_min_df = \
rolling_player_min_df.groupby('element', as_index=True)[['element_fixture_rank'] + rolling_cols]\
.rolling(38, min_periods=1).min().reset_index('element')

# increment element fixture rank, as we don't know data ahead of time
rolling_player_min_df['element_fixture_rank'] =\
rolling_player_min_df.groupby('element')['element_fixture_rank'].rank() + 1

rolling_player_min_df.head(10)

,element,element_fixture_rank,total_points
0,1,7.0,3.0
1,1,7.0,3.0
2,1,7.0,3.0
3,1,7.0,1.0
4,1,7.0,1.0
5,1,7.0,1.0
6,1,7.0,1.0
7,1,7.0,0.0
8,1,7.0,0.0
9,1,7.0,0.0


In [19]:
element_gameweek_df =\
    element_gameweek_df.join(
        rolling_player_min_df.set_index(['element', 'element_fixture_rank']),
        on=['element', 'element_fixture_rank'],
        rsuffix='_player_min',
        how='left'
)

element_gameweek_df.head()

,element,event,own_team,opposition_team,was_home,element_type,value,total_points,minutes,element_fixture_rank,total_points_player_mean,total_points_player_max,total_points_player_min
0,1,1,1,13,1,1,50,3,90,1.0,NaN,NaN,NaN
1,1,2,1,6,0,1,50,3,90,2.0,3.0,3.0,NaN
2,1,3,1,19,1,1,50,3,90,3.0,3.0,3.0,NaN
3,1,4,1,5,0,1,50,1,90,4.0,3.0,3.0,NaN
4,1,5,1,15,0,1,50,2,90,5.0,2.5,3.0,NaN


In [20]:
# what should we do about gameweek 1, where we don't have an average of points?
# does this work for double or missing gameweeks?

## Get position mean points

In [21]:
# copy main df
rolling_position_mean_df = element_gameweek_df.copy()

# filter out players that did not play
rolling_position_mean_df = rolling_position_mean_df[rolling_position_mean_df['minutes'] > 0]
# get average points by poisition by event
rolling_position_mean_df = \
rolling_position_mean_df.groupby(['element_type', 'event'])['total_points'].mean().reset_index()


# average rolling cols over previous events 
rolling_position_mean_df = \
rolling_position_mean_df.groupby('element_type', as_index=True)[['event'] + rolling_cols]\
.rolling(38, min_periods=1).mean().reset_index('element_type')

# # increment element fixture rank, as we don't know data ahead of time
rolling_position_mean_df['event'] =\
rolling_position_mean_df.groupby('element_type')['event'].rank() + 1

rolling_position_mean_df.head()

,element_type,event,total_points
0,1,2.0,4.400000
1,1,3.0,4.000000
2,1,4.0,3.682540
3,1,5.0,3.761905
4,1,6.0,3.509524


In [22]:
element_gameweek_df =\
    element_gameweek_df.join(
        rolling_position_mean_df.set_index(['element_type', 'event']),
        on=['element_type', 'event'],
        rsuffix='_position_mean',
        how='left'
)

element_gameweek_df.head()

,element,event,own_team,opposition_team,was_home,element_type,value,total_points,minutes,element_fixture_rank,total_points_player_mean,total_points_player_max,total_points_player_min,total_points_position_mean
0,1,1,1,13,1,1,50,3,90,1.0,NaN,NaN,NaN,NaN
1,1,2,1,6,0,1,50,3,90,2.0,3.0,3.0,NaN,4.400000
2,1,3,1,19,1,1,50,3,90,3.0,3.0,3.0,NaN,4.000000
3,1,4,1,5,0,1,50,1,90,4.0,3.0,3.0,NaN,3.682540
4,1,5,1,15,0,1,50,2,90,5.0,2.5,3.0,NaN,3.761905


In [23]:
# what should we do about gameweek 1, where we don't have an average of points?

# Modelling data

## Splitting data

In [197]:
# train data on previous weeks and predict current week
train_df = element_gameweek_df.copy()
test_df = element_gameweek_df.copy()

train_df = train_df[train_df['event'] < current_event]
test_df = test_df[test_df['event'] == current_event]

In [198]:
formula = \
'total_points ~ C(element) + C(element_type) + C(own_team) + value + C(opposition_team) + was_home'

In [199]:
y, X = patsy.dmatrices(formula, element_gameweek_df, return_type='dataframe')
y_train, X_train = patsy.dmatrices(formula, train_df, return_type='dataframe')
y_test, X_test = patsy.dmatrices(formula, test_df, return_type='dataframe')

In [200]:
def add_missing_columns(df, columns):
    for col in set(columns) - set(df.columns):
        df[col] = np.zeros(len(df))
    
    return df[columns]

In [201]:
X_train = add_missing_columns(X_train, X.columns)
X_test = add_missing_columns(X_test, X.columns)

In [202]:
# should we scale the variables? let's find out

## Building model

In [203]:
# instantiate model
model = Sequential()

# add the input layer, i.e. first hidden layer
model.add(Dense(units=25, activation='relu', input_shape=(len(X.columns),)))

# add hidden layers
model.add(Dense(units=25, activation='relu'))
# model.add(Dense(units=25, activation='relu'))
# model.add(Dense(units=25, activation='relu'))

# add the output layer
model.add(Dense(units=1, activation='linear'))

In [204]:
# set training configuration
model.compile(
    loss='mean_squared_error',
    optimizer='adam'
)

In [205]:
# train model on training set
model.fit(
    X_train, # explanatory variable training data
    y_train, # response variable training data
    epochs=20, # number of training iterations
    batch_size=10, # The batch size that you specify in the code above defines the number of samples that going to be propagated through the network
    verbose=1 # monitor training progress
)

Epoch 1/20
5370/5370 [==============================] - 4s 765us/sample - loss: 6.4361
Epoch 2/20
5370/5370 [==============================] - 1s 201us/sample - loss: 5.9749
Epoch 3/20
5370/5370 [==============================] - 1s 196us/sample - loss: 5.3933
Epoch 4/20
5370/5370 [==============================] - 1s 191us/sample - loss: 4.9448
Epoch 5/20
5370/5370 [==============================] - 1s 198us/sample - loss: 4.6440
Epoch 6/20
5370/5370 [==============================] - 1s 195us/sample - loss: 4.5369
Epoch 7/20
5370/5370 [==============================] - 1s 201us/sample - loss: 4.4531
Epoch 8/20
5370/5370 [==============================] - 1s 201us/sample - loss: 4.3696
Epoch 9/20
5370/5370 [==============================] - 1s 194us/sample - loss: 4.3750
Epoch 10/20
5370/5370 [==============================] - 1s 200us/sample - loss: 4.3708
Epoch 11/20
5370/5370 [==============================] - 1s 197us/sample - loss: 4.2940
Epoch 12/20
5370/5370 [==================

## Evaluating model

In [206]:
# get predictions for test set
y_pred = model.predict(X_test).flatten()

# calculate cross entropy loss for test set
test_loss = (mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred))
test_loss

(1.392820210538127, 5.061308579334623)

In [207]:
pd.Series(y_pred).describe()

count    550.000000
mean       1.688168
std        1.755780
min       -0.106717
25%        0.244827
50%        1.075141
75%        2.593533
max        8.939925
dtype: float64

In [142]:
# current event = 11
# single layer, epochs=20, batch_size=10, units=25, activation='relu'
(1.4232961570169764, 5.213269991856961)
(1.4267152123896152, 5.127159289088593)
(1.2716270690303175, 4.645559092986532)
(1.3744119672792359, 4.794474121162641)
(1.4836523056724593, 5.1867786737338255)
(1.4581301436362302, 5.238504891337133)

# single layer, epochs=20, batch_size=10, units=50, activation='relu'
(1.356415465934783, 4.967106174118941)
(1.2414628361688933, 4.651043010541483)
(1.295564303908991, 4.751978566815723)
(1.2971654627538924, 4.6155169958606495)
(1.3823382342334396, 4.92160634621547)

# single layer, epochs=20, batch_size=10, units=75, activation='relu'
(1.255836346383644, 4.549992267627301)
(1.260702874808569, 4.564730831558708)
(1.4674110345035043, 5.174372475200111)
(1.3057418049923994, 4.551337821245331)
(1.2302483491746434, 4.5812340773162195)

# single layer, epochs=20, batch_size=10, units=100, activation='relu'
(1.3028445945665454, 4.702209251759358)
(1.4917569621783218, 5.470631916839363)
(1.3311773778296554, 4.726627514798753)
(1.297806473702566, 4.706734335626567)
(1.2320183746943978, 4.592257904492718)

# single layer, epochs=20, batch_size=10, units=125, activation='relu'
(1.3136140566359285, 4.7713821610196145)
(1.2762708149977215, 4.662478514197314)
(1.4199238726769083, 4.937220011507264)
(1.2830597889546211, 4.682419397917666)
(1.2701970337619704, 4.740322420836487)

# single layer, epochs=20, batch_size=10, units=150, activation='relu'
(1.2780502292137312, 4.427330559316878)
(1.312058645841335, 4.681500513090768)
(1.2621182357645377, 4.744487518877087)

# single layer, epochs=20, batch_size=10, units=200, activation='relu'
(1.4079818554392824, 4.921251557293539)
(1.4020005321016662, 5.052188755142338)
(1.5076904161015399, 5.2527472285245596)



# two layer, epochs=20, batch_size=10, units=(75,75), activation='relu'
(1.4362596557024987, 5.1510562807565945)
(1.3887610258968501, 5.2384691130959125)
(1.3232729913513293, 5.0331249719374025)

(1.4199238726769083, 4.937220011507264)